In [ ]:
#os.chdir('/content/drive/MyDrive/Escuela/Doctorado/Projects/ConvNext/ConvNext_V2')

In [ ]:
!nvidia-smi

In [ ]:
import torch
torch.cuda.empty_cache()
torch.cuda.memory_allocated()  
torch.cuda.memory_reserved()  
import zipfile
import os
import nibabel as nib
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F
from torch.utils.data import random_split
import wandb
from torchvision import models
import torch.optim as optim
#new 
#import albumentations as A
#from albumentations.pytorch import ToTensorV2

# DataLoader

In [ ]:
%run src/ConvNext_models.ipynb
%run src/metrics.ipynb
%run src/hg_lossfunctions.ipynb
%run src/trainer.ipynb
%run src/BratsDataloader.ipynb
%run src/Optimizer.ipynb
%run src/UNET.ipynb

In [ ]:
base_path ="../datasets/BRATS2021/"#"/content/drive/MyDrive/datasets/BrATS2023/BrATS2023/"#"BraTS2021_Training_Data" #"BrATS2023" #"/content/drive/MyDrive/datasets/BrATS2023/BrATS2023"

transform = transforms.Compose([
    transforms.Resize((512, 512)), 
])

mask_transform = transforms.Compose([
    transforms.Resize((512, 512)) 
])


# Crear el dataset y el DataLoader
dataset = BRATSDataset_2(base_path, img_transform=transform, mask_transform = mask_transform)

train_size = int(0.7 * len(dataset))  # 70% para entrenamiento
val_size   = int(0.15 * len(dataset))   # 15% para validación
test_size  = len(dataset) - train_size - val_size  # 15% para prueba

# Dividir el dataset en entrenamiento, validación y prueba
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Crear los DataLoaders para cada conjunto
BATCH_SIZE = 6
train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size   = BATCH_SIZE, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size  = BATCH_SIZE, shuffle=False)
print(np.shape(train_loader))
for img, mask in train_loader:
    print(np.shape(img))
    print(np.shape(mask))
    break
# IN testing pahse
def get_model_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    return total_params

# 2. Obtener el tamaño en memoria del modelo (sin necesidad de guardarlo en archivo)
def get_model_memory_size(model):
    param_size = sum(p.numel() * p.element_size() for p in model.parameters())
    return param_size
def get_model_flops(model, input_size=(1, 512, 512), device='cuda'):
    model = model.to(device)
    model.eval()
    
    total_flops = 0
    total_params = sum(p.numel() for p in model.parameters())

# Training

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

device          = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes     = 1  # Ejemplo: edema, realce, necrosis, etc.
ConvNext_models = {  #'HyperNetSegmentation'    : HyperNetSegmentation,
                     'ConvNeXtSegmentationMF'  : ConvNeXtSegmentationMF, # dió IoU: 0.8
                     #'ConvNeXtUNet'            : ConvNeXtUNet,
                     #'ConvNeXtUNet_B'          : ConvNeXtUNet_B,
                     #ConvNext_Base'           : ConvNeXtSegmentation
                     #'ConvNeXtSegmentationRA'  : ConvNeXtSegmentationRA #use highg resources
}

optimizers      =  { 
                     #'Adam'     : lambda model: optim.Adam(model.parameters(), lr = 0.001),
                     'AdamW'    : lambda model: optim.AdamW(model.parameters(), lr=0.001),
                     #'CRTAdamW' : lambda model:CRTAdamW(model.parameters(), lr = 0.001),
                  }

functions_losses = { 'BCEDiceLoss': BCEDiceLoss(),
                     #'DSC': DiceLoss(),
                     'BCE':nn.BCEWithLogitsLoss()
                   }

for model_name, model_class in ConvNext_models.items():      # Models 
    for optimizer_name, optimizer_class in optimizers.items(): # Optimizers
        for losses_name, losses_class in functions_losses.items(): # Losses
            print('------------------------ Model Name:', model_name)
            print('------------------------ OPtimizer Name:', optimizer_name)
            print('------------------------ Loss Name:', losses_name)

            if model_name =='ConvNeXtUNet' or  model_name == 'ConvNeXtRAUNet' or model_name == 'ConvNeXtMFRAUNet':
                model = model_class(num_classes, backbone_type='base')
            else:
                model       =  model_class(num_classes)   #ConvNeXtSegmentation(num_classes)
            
            model       = model.to(device)  # Si estás usando una GPU
            criterion   =  losses_class #nn.BCEWithLogitsLoss()  # CrossEntropyLoss()
            optimizer   =  optimizer_class(model)# CRTAdamW(model.parameters(), lr = 0.001)#optim.AdamW(model.parameters(), lr=0.001)# CRTAdamW
            num_epochs  = 500
        
            Metrics = []
            Conv_Next_trainer = UNetTrainer(
                    model        = model,
                    device       = device,
                    optimizer    = optimizer,
                    criterion    = criterion,
                    train_loader = train_loader,
                    val_loader   = val_loader,
                    test_loader  = test_loader,
                    epochs       = num_epochs,
                    filename_model = model_name + '.pth'
                )
            total_params      = get_model_parameters(model)
            model_memory_size = get_model_memory_size(model)/ (1024 ** 2)
            FLOps             = get_model_flops(model)
            print(FLOps)
            wandb.init(project="ConvNext_V2_RTX4070Super_Brats2024_Round1", config={
                'Dataset'       : base_path, 
                'device'        : device,
                'model name'    : model_name,
                'epochs'        : num_epochs,
                'batch size'    : BATCH_SIZE,
                'loss_function' : losses_name, #criterion,
                'optimizer'     : optimizer_name,#optimizer,
                'total params'  :total_params,
                'Model Size (MB)'   : model_memory_size
            })
            # Start training
            Conv_Next_trainer.train()
            # Test the model if needed
            L_Net_images, L_Net_preds, L_Net_targets = Conv_Next_trainer.test()
            # Visualize results
            final_file = "BRATS2024_"+model_name+"_"+ optimizer_name+"_"+losses_name+'.pdf'
            Conv_Next_trainer.visualize_results(L_Net_images, L_Net_targets, L_Net_preds, num_samples=10, save_path= final_file)
            for th in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]:
                metrics = Conv_Next_trainer.evaluate_metrics(threshold=th)
                Metrics.append(metrics)
        
            wandb.finish()

# Traiing using k-Fold

In [ ]:

device          = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes     = 1  # Ejemplo: edema, realce, necrosis, etc.
ConvNext_models = {  #'UNET'    : UNet,
                     'ConvNeXtSegmentationMF'  : ConvNeXtSegmentationMF#, # dió IoU: 0.8
                     #'ConvNeXtUNet'            : ConvNeXtUNet,
                     #'ConvNeXtUNet_B'          : ConvNeXtUNet_B,
                     #'ConvNext_Base'           : ConvNeXtSegmentation
                     #'ConvNeXtSegmentationRA'  : ConvNeXtSegmentationRA #use highg resources
}

optimizers      =  { 
                     #'Adam'     : lambda model: optim.Adam(model.parameters(), lr = 0.001),
                     'AdamW'    : lambda model: optim.AdamW(model.parameters(), lr=0.001),
                     #'CRTAdamW' : lambda model:CRTAdamW(model.parameters(), lr = 0.001),
                    }

functions_losses = { #'BCEDiceLoss': BCEDiceLoss(),
                     #'DSC': DiceLoss(),
                     'BCE':nn.BCEWithLogitsLoss()
                   }

# VIolin plot

In [ ]:
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, Subset
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import wandb

def train_models_violin(
    ConvNext_models,
    optimizers,
    functions_losses,
    train_loader,
    val_loader,
    test_loader,
    num_classes=1,
    device=None,
    num_epochs=500,
    BATCH_SIZE=6,
    use_kfold=False,
    k=5,
    metrics_to_plot=None   # <- lista opcional de métricas a graficar
):
    device = device or torch.device("cuda" if torch.cuda.is_available() else "cpu")
    dataset = train_loader.dataset

    all_metrics_data = []

    for model_name, model_class in ConvNext_models.items():
        for optimizer_name, optimizer_class in optimizers.items():
            for losses_name, losses_class in functions_losses.items():
                
                print('------------------------ Model:', model_name)
                print('------------------------ Optimizer:', optimizer_name)
                print('------------------------ Loss:', losses_name)

                metrics_per_model = []

                if use_kfold:
                    kf = KFold(n_splits=k, shuffle=True, random_state=42)
                    fold_iter = enumerate(kf.split(dataset))
                else:
                    fold_iter = [(0, (range(len(dataset)), range(len(val_loader.dataset))))]

                for fold, (train_idx, val_idx) in fold_iter:
                    print(f"===== Fold {fold+1} =====")

                    if use_kfold:
                        train_subset = Subset(dataset, train_idx)
                        val_subset   = Subset(dataset, val_idx)
                        train_loader_fold = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True)
                        val_loader_fold   = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False)
                    else:
                        train_loader_fold = train_loader
                        val_loader_fold   = val_loader

                    # Instanciar modelo
                    if model_name in ['ConvNeXtUNet', 'ConvNeXtRAUNet', 'ConvNeXtMFRAUNet']:
                        model = model_class(num_classes, backbone_type='base')
                    else:
                        model = model_class(num_classes)
                    model = model.to(device)

                    criterion = losses_class
                    optimizer = optimizer_class(model)

                    trainer = UNetTrainer(
                        model        = model,
                        device       = device,
                        optimizer    = optimizer,
                        criterion    = criterion,
                        train_loader = train_loader_fold,
                        val_loader   = val_loader_fold,
                        test_loader  = test_loader,
                        epochs       = num_epochs,
                        filename_model = f"{model_name}_fold{fold+1}_2021.pth"
                    )

                    wandb.init(project="ConvNext_V2_Kfold_RTX4070Super_Round1", config={
                        'Dataset'       : 'CustomDataset', 
                        'device'        : device,
                        'model name'    : model_name,
                        'epochs'        : num_epochs,
                        'batch size'    : BATCH_SIZE,
                        'loss_function' : losses_name,
                        'optimizer'     : optimizer_name
                    })

                    trainer.train()
                    L_Net_images, L_Net_preds, L_Net_targets = trainer.test()
                    # Visualize results
                    final_file = model_name+"_"+ optimizer_name+"_"+losses_name+"_k_"+ str(fold)+'.pdf'
                    trainer.visualize_results(L_Net_images, L_Net_targets, L_Net_preds, num_samples=5)
                    metrics_fold = trainer.evaluate_metrics(threshold=0.4)
                    print("Metrics Fold: ", metrics_fold )
                    # Guardar métricas
                    for metric_name, value in metrics_fold.items():
                        metrics_per_model.append({
                            "Model": model_name,
                            "Optimizer": optimizer_name,
                            "Loss": losses_name,
                            "Fold": fold+1,
                            "Metric": metric_name,
                            "Value": value
                        })
                        all_metrics_data.append({
                            "Model": model_name,
                            "Optimizer": optimizer_name,
                            "Loss": losses_name,
                            "Fold": fold+1,
                            "Metric": metric_name,
                            "Value": value
                        })

                    wandb.finish()

                # Crear DataFrame solo para este modelo y configuración
                df_model = pd.DataFrame(metrics_per_model)

                # Filtrar solo métricas deseadas si se pasa la lista
                if metrics_to_plot is not None:
                    df_model = df_model[df_model['Metric'].isin(metrics_to_plot)]

                
                plt.rcParams.update({
                "axes.titlesize": 20,      # título
                "axes.labelsize": 24,      # etiquetas de ejes
                "xtick.labelsize": 21,     # ticks X
                "ytick.labelsize": 21,     # ticks Y
                "legend.title_fontsize": 21,
                "legend.fontsize": 23
                })
                name_map = {
                    "Dice Coefficient"     : "DSC",
                    "IoU Score"            : "mIoU",
                    "Sensitivity (Recall)" : "Sensitivity",
                    "Specificity"          : "Specificity",
                    "Accuracy"             : "Accuracy",
                    "F1-Score"             : "F1-score"
                }
                
                df_model["Metric"] = df_model["Metric"].replace(name_map)
                
                order = ["DSC", "IoU", "Recall", "Specificity", "Accuracy", "F1"]
                fig, ax = plt.subplots(figsize=(10,7))#plt.figure(figsize=(24,12))
                sns.boxplot(
                    x="Metric", y="Value", hue="Loss",   # opcional: hue="Optimizer"
                    data  = df_model,
                    #notch = True # for IC 95
                    #order=order  # <- opcional

                )
                #sns.violinplot(
                #    x     = "Metric",
                #    y     = "Value",
                #    hue   = "Loss",   # opcional: hue="Optimizer"
                #    data  = df_model,
                #    split = True,
                #    inner = None, #quartile"
                #    bw    = 0.5,           # ajusta el ancho del KDE (kernel density)
                #    cut   = 0
                #)
                ax.set_ylim(0.0, 1.0)                          # límites de 0 a 1
                ax.set_yticks(np.arange(0.0, 1.01, 0.1))       # ticks cada 0.1
                ax.set_title(f"Metric distribution for {model_name} ({optimizer_name}, {losses_name})")
                ax.set_ylabel("Metric value")
                ax.set_xlabel("Metrics")
                ax.grid(True)
                plt.setp(ax.get_xticklabels(), rotation=0)
                ax.legend(title="Function Loss")
                
                boxplot_file = f"{model_name}_{optimizer_name}_{losses_name}.pdf"
                plt.savefig(boxplot_file, dpi=300, bbox_inches='tight')
                plt.show()
                #ax.set_ylim(0.0, 1.0)
                #ax.set_yticks(np.arange(0.0, 1.01, 0.1))
                #plt.title(f"Metric distribution for {model_name} ({optimizer_name}, {losses_name})")
                #plt.ylabel("Metric value")
                #plt.xlabel("Metrics")
                #plt.xticks(rotation = 0)
                #plt.legend(title="Function Loss")
                #boxplot_file = model_name +"_"+ optimizer_name+"_"+losses_name +'.pdf'
                #plt.savefig(boxplot_file, dpi=300, bbox_inches='tight')  # opcional: transparent=True, format="png" 
                #plt.show()

    return pd.DataFrame(all_metrics_data)



In [ ]:
df_metrics = train_models_violin(
    ConvNext_models  = ConvNext_models,
    optimizers       = optimizers,
    functions_losses = functions_losses,
    train_loader     = train_loader,
    val_loader       = val_loader,
    test_loader      = test_loader,
    num_classes      = 1,
    device           = device,
    num_epochs       = 100,
    BATCH_SIZE       = BATCH_SIZE,
    use_kfold        = False,
    k                = 1,
    metrics_to_plot  = ["IoU Score","Dice Coefficient", "Sensitivity (Recall)", "Specificity",'Accuracy','F1-Score']
)

In [ ]:
#df_metrics.to_csv("ConvNeXt_all_metrics_BrATS2021_3_with_UNET_Brats2023.csv")